In [ ]:
%cd /content/drive/MyDrive/Đồ Án Thu Thập Tiền Xử Lý/code

/content/drive/MyDrive/Đồ Án Thu Thập Tiền Xử Lý/code


In [ ]:
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 56.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Embedding, Dropout, Dense, Input, LSTM, Bidirectional, Conv1D,  GlobalMaxPooling1D, GlobalAveragePooling1D,  Concatenate, SpatialDropout1D, Activation, Flatten, concatenate
from keras.preprocessing import text, sequence
from keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import regularizers

In [ ]:
TRAIN_DIR = 'data/train.csv'
VAL_DIR   = 'data/dev.csv'
TEST_DIR  = 'data/test.csv'

#1. Data Preparation

In [ ]:
train = pd.read_csv(TRAIN_DIR)
test = pd.read_csv(TEST_DIR)
dev = pd.read_csv(VAL_DIR)

In [ ]:
def read_file(df):
    X = df.pop('review')
    y = df.replace({np.nan: 0,
                    'Positive': 1,
                    'Negative': 2,
                    'Neutral': 3}).astype(np.uint8)

    print('X.shape:', X.shape, 'y.shape:', y.shape)
    return X, y

In [ ]:
aspects = ['Hotel#General',
          'Hotel#Price',
          'Hotel#Designs_Features',
          'Hotel#Cleanliness',
          'Hotel#Comfort',
          'Hotel#Quality',
          'Hotel#Miscellaneous',
          'Rooms#General',
          'Rooms#Price',
          'Rooms#Designs_Features',
          'Rooms#Cleanliness',
          'Rooms#Comfort',
          'Rooms#Quality',
          'Rooms#Miscellaneous',
          'Room_amenities#General',
          'Room_amenities#Price',
          'Room_amenities#Designs_Features',
          'Room_amenities#Cleanliness',
          'Room_amenities#Comfort',
          'Room_amenities#Quality',
          'Room_amenities#Miscellaneous',
          'Facilities#General',
          'Facilities#Price',
          'Facilities#Designs_Features',
          'Facilities#Cleanliness',
          'Facilities#Comfort',
          'Facilities#Quality',
          'Facilities#Miscellaneous',
          'Service#General',
          'Location#General',
          'Food_Drinks#Price',
          'Food_Drinks#Quality',
          'Food_Drinks#Style_Options',
          'Food_Drinks#Miscellaneous'
          ]

replacements = {0: None, 1: 'positive', 2: 'negative', 3: 'neutral'}

In [ ]:
X_train, y_train = read_file(train)
X_dev,   y_dev   = read_file(dev)
X_test,  y_test  = read_file(test)

X.shape: (4805,) y.shape: (4805, 34)
X.shape: (1602,) y.shape: (1602, 34)
X.shape: (1602,) y.shape: (1602, 34)


In [ ]:
def make_outputs(df):
    outputs = []
    for row in range(len(df)):
        row_one_hot = []
        for col in range(0, len(df.columns)):
            sentiment = df.iloc[row, col]
            if   sentiment == 0: one_hot = [1, 0, 0, 0] # None
            elif sentiment == 1: one_hot = [0, 1, 0, 0] # Pos
            elif sentiment == 2: one_hot = [0, 0, 1, 0] # Neg
            elif sentiment == 3: one_hot = [0, 0, 0, 1] # Neu
            row_one_hot.append(one_hot)
        outputs.append(row_one_hot)
    outputs = np.array(outputs, dtype='uint8')
    return outputs.reshape(len(df), -1)

#2. Text Preprocess


## 2.1. Chuẩn hóa Unicode tiếng Việt

In [ ]:
import regex as re

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()

# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

## 2.2. Chuấn hóa kiểu gõ dấu

In [ ]:
import re
import os
import sys
import pandas as pd
# from Logger import LogEventSourcing
from datetime import datetime
import dateutil.parser
import traceback
import time
import requests


# logger = LogEventSourcing()

def call_api(data, url, method, timeout=3):
    headers = {
        'content-type': "application/x-www-form-urlencoded",
        'cache-control': "no-cache",
        'postman-token': "6a410524-a8e2-79c7-bd9d-53e4b68c84c7"
    }
    response = requests.request(method, url, data=data, headers=headers, timeout=timeout)
    return response


uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()


def convertwindown1525toutf8(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

"""
    Start section: Chuyển câu văn về kiểu gõ telex khi không bật Unikey
    Ví dụ: thủy = thuyr, tượng = tuwowngj
"""
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)


def vn_word_to_telex_type(word):
    dau_cau = 0
    new_word = ''
    for char in word:
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            new_word += char
            continue
        if y != 0:
            dau_cau = y
        new_word += bang_nguyen_am[x][-1]
    new_word += bang_ky_tu_dau[dau_cau]
    return new_word


def vn_sentence_to_telex_type(sentence):
    """
    Chuyển câu tiếng việt có dấu về kiểu gõ telex.
    :param sentence:
    :return:
    """
    words = sentence.split()
    for index, word in enumerate(words):
        words[index] = vn_word_to_telex_type(word)
    return ' '.join(words)


"""
    End section: Chuyển câu văn về kiểu gõ telex khi không bật Unikey
"""

"""
    Start section: Chuyển câu văn về cách gõ dấu kiểu cũ: dùng òa úy thay oà uý
    Xem tại đây: https://vi.wikipedia.org/wiki/Quy_t%E1%BA%AFc_%C4%91%E1%BA%B7t_d%E1%BA%A5u_thanh_trong_ch%E1%BB%AF_qu%E1%BB%91c_ng%E1%BB%AF
"""


def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        words[index] = chuan_hoa_dau_tu_tieng_viet(word)
    return ' '.join(words)


"""
    End section: Chuyển câu văn về cách gõ dấu kiểu cũ: dùng òa úy thay oà uý
    Xem tại đây: https://vi.wikipedia.org/wiki/Quy_t%E1%BA%AFc_%C4%91%E1%BA%B7t_d%E1%BA%A5u_thanh_trong_ch%E1%BB%AF_qu%E1%BB%91c_ng%E1%BB%AF
"""
if __name__ == '__main__':
    # with open('C:/Users/htv/Desktop/testunicode.txt') as f:
    #     content = f.read()
    #     output = decodetounicode(content)
    #     wirtefile('C:/Users/htv/Desktop/unicode.txt', output)
    txt = 'oà uý'
    # print(is_valid_vietnam_word(txt))
    txt = chuan_hoa_dau_cau_tieng_viet(txt)
    print(txt)

òa úy


## 2.3. Chuẩn hoá teencode (Read teencode.xlsx)

In [ ]:
teencode = pd.read_excel("/content/drive/MyDrive/Đồ Án Thu Thập Tiền Xử Lý/code/teencode.xlsx")

def chuan_hoa_teen_code(sentence):
    result = [x.strip() for x in sentence.split()]
    for i in range(0, len(result)):
        for j in range(0, len(teencode)):
            if (result[i] == teencode.at[j, "code"]):
                result[i] = teencode.at[j, "chuanhoa"]
    x = " ".join(result)
    x.strip()
    return x

##2.4. Tách từ tiếng việt sử dụng Underthesea

In [ ]:
!pip install underthesea
from underthesea import word_tokenize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##2.5. Loại bỏ Stop Word (Read Stopword.txt)

In [ ]:
with open("/content/drive/MyDrive/Đồ Án Thu Thập Tiền Xử Lý/code/Stopword.txt",'rb') as f:
    contents = f.read()
contents = contents.decode("utf-16")
contents = contents.split("\r\n")
stopword = set(contents)

def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

##2.6. Xử lý tổng hợp

In [ ]:
def text_preprocess(document):
    # đưa về lower
    document = document.lower()
    # chuẩn hóa unicode
    document = covert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = chuan_hoa_dau_cau_tieng_viet(document)
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    # chuẩn hoá teencode
    document = chuan_hoa_teen_code(document)

    # # tách từ
    # document = word_tokenize(document, format="text")
    return document

In [ ]:
text_preprocess("Khách sạn nằm ở vị trí gần trung tâm, thuận tiện đi lại, tham quan các địa điểm nổi tiếng. \n Phòng rộng rãi, ấm cúng, sạch sẽ, view đẹp nhìn ra sân vườn.")

'khách sạn nằm ở vị trí gần trung tâm thuận tiện đi laị tham quan các địa điểm nổi tiếng phòng rộng raĩ ấm cúng sạch sẽ view đẹp nhìn ra sân vườn'

# 3. Train Model

##Text Preprocess

In [ ]:
X_train_pr = X_train.apply(text_preprocess)
X_test_pr = X_test.apply(text_preprocess)
X_dev_pr = X_dev.apply(text_preprocess)

In [ ]:
EMBEDDING_FILE = 'cc.vi.300.vec'
embedding_index = {}

with open(EMBEDDING_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.strip().split(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

print('Tổng số từ vựng trong không gian nhúng: ', len(embedding_index))

Tổng số từ vựng trong không gian nhúng:  2000000


In [ ]:
MAX_FEATURES = 5000
MAXLEN = 150
EMBED_SIZE = 300

In [ ]:
y_train_model = make_outputs(y_train)
y_dev_model = make_outputs(y_dev)
y_test_model = make_outputs(y_test)

In [ ]:
y_train_model.shape

(4805, 136)

In [ ]:
tokenizer = text.Tokenizer(num_words = MAX_FEATURES, lower=True)
tokenizer.fit_on_texts(list(X_train))

In [ ]:
X_train_pad = tokenizer.texts_to_sequences(X_train_pr)
X_test_pad = tokenizer.texts_to_sequences(X_test_pr)
X_dev_pad = tokenizer.texts_to_sequences(X_dev_pr)

In [ ]:
X_train_pad = pad_sequences(X_train_pad, maxlen = MAXLEN)
X_test_pad = pad_sequences(X_test_pad, maxlen = MAXLEN)
X_dev_pad = pad_sequences(X_dev_pad, maxlen = MAXLEN)

In [ ]:
word_index = tokenizer.word_index

embedding_matrix = np.zeros((MAX_FEATURES, EMBED_SIZE))

for word, i in word_index.items():
    if i >= MAX_FEATURES:
        continue

    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
#training params
BATCH_SIZE = 32
EPOCHS =15

#model parameters
num_filters = 64
EMBED_DIM = 300
WEIGHT_DECAY = 1e-4

In [ ]:
embedding_matrix.shape

(5000, 300)

## Model LSTM

In [ ]:
inputs = Input(shape=(MAXLEN,))

x = Embedding(MAX_FEATURES, EMBED_SIZE, weights=[embedding_matrix], trainable=True)(inputs)
x = SpatialDropout1D(0.35)(x)

x = LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15)(x)
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

output_layer = Concatenate(axis=1)([Dense(units=4, activation='softmax', name=aspect.replace('#', '-').replace('&', '_'))(x) for aspect in aspects])

model_ltsm = Model(inputs, output_layer)
model_ltsm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose=1, patience=3)


history = model_ltsm.fit(x = X_train_pad, y = y_train_model,
                    batch_size=32,
                    epochs=15,
                    verbose=1,
                    validation_data=(X_dev_pad, y_dev_model),
                    callbacks=[early_stopping])

Epoch 1/15
151/151 [==============================] - 179s 972ms/step - loss: 0.1770 - accuracy: 2.0812e-04 - val_loss: 0.1451 - val_accuracy: 0.0000e+00
Epoch 2/15
151/151 [==============================] - 134s 887ms/step - loss: 0.1321 - accuracy: 0.0000e+00 - val_loss: 0.1238 - val_accuracy: 0.0000e+00
Epoch 3/15
151/151 [==============================] - 127s 840ms/step - loss: 0.1110 - accuracy: 0.0000e+00 - val_loss: 0.1045 - val_accuracy: 0.0000e+00
Epoch 4/15
151/151 [==============================] - 119s 791ms/step - loss: 0.0963 - accuracy: 0.0000e+00 - val_loss: 0.0966 - val_accuracy: 0.0000e+00
Epoch 5/15
151/151 [==============================] - 121s 804ms/step - loss: 0.0858 - accuracy: 0.0000e+00 - val_loss: 0.0901 - val_accuracy: 0.0000e+00
Epoch 6/15
151/151 [==============================] - 123s 814ms/step - loss: 0.0780 - accuracy: 0.0000e+00 - val_loss: 0.0852 - val_accuracy: 0.0000e+00
Epoch 7/15
151/151 [==============================] - 121s 803ms/step - loss

In [ ]:
model_ltsm.save('result_model/model_lstm_v2.h5')

In [ ]:
y_pred = model_ltsm.predict(X_test_pad)
y_pred = y_pred.reshape(len(y_pred), -1, 4)
y_pred_argmax = np.argmax(y_pred, axis=-1)

51/51 [==============================] - 3s 49ms/step


In [ ]:
vlsp_results = []
for index, pred in enumerate(y_pred_argmax):
    sentiments = map(lambda x: replacements[x], pred)
    result = {
        'id': f'#{index + 1}',
        'text': X_test[index],
        'acsa': []
    }
    for aspect, sentiment in zip(aspects, sentiments):
        if sentiment: result['acsa'].append('{'+ aspect + ', ' + sentiment + '}')
    vlsp_results.append(result)
vlsp_results[:3]

[{'id': '#1',
  'text': 'Khách sạn tuyệt vời, mình ở 3n2đ đi chơi không kịp báo dọn phòng nhưng khách sạn đã dọn giúp sạch sẽ tinh tươm trước khi mình về, mình book được giá tốt nữa. Rất hài lòng, nv nhiệt tình dễ thương nhẹ nhàng, có dịp quay lại mình sẽ ở lại đây.',
  'acsa': ['{Hotel#General, positive}',
   '{Hotel#Comfort, positive}',
   '{Rooms#Cleanliness, positive}',
   '{Service#General, positive}']},
 {'id': '#2',
  'text': 'Nhân viên phục vụ dễ thương! Khuyết điểm là khách sạn ít đèn quá tối, rất khó chịu. Vòi sen bồn rửa mặt ngắn quá, rửa mặt xong nước chảy tràn ra tùm lum. Phòng tắm không có bồn. Ăn sáng tạm ổn, nói chung với mức tiền đó thì chấp nhận được',
  'acsa': ['{Room_amenities#Designs_Features, negative}',
   '{Room_amenities#Quality, negative}',
   '{Service#General, positive}',
   '{Food_Drinks#Style_Options, negative}']},
 {'id': '#3',
  'text': 'Khách sạn rất gần chợ đêm. Xung quanh khách sạn có nhiều địa điểm ăn uống kể cả ban ngày và ban đêm. Nhân viên lễ tân

In [ ]:
with open('result_data/model_LSTM_v2.txt', 'w', encoding='utf-8') as f:
    for result in vlsp_results:
        id, text, acsa = result.values()
        f.write(f"{id}\n{text}\n{', '.join(acsa)}\n\n")

Save dev

In [ ]:
y_pred = model_ltsm.predict(X_dev_pad)
y_pred = y_pred.reshape(len(y_pred), -1, 4)
y_pred_argmax = np.argmax(y_pred, axis=-1)

51/51 [==============================] - 3s 50ms/step


In [ ]:
vlsp_results = []
for index, pred in enumerate(y_pred_argmax):
    sentiments = map(lambda x: replacements[x], pred)
    result = {
        'id': f'#{index + 1}',
        'text': X_test[index],
        'acsa': []
    }
    for aspect, sentiment in zip(aspects, sentiments):
        if sentiment: result['acsa'].append('{'+ aspect + ', ' + sentiment + '}')
    vlsp_results.append(result)

with open('result_data/model_dev_LSTM_v2.txt', 'w', encoding='utf-8') as f:
    for result in vlsp_results:
        id, text, acsa = result.values()
        f.write(f"{id}\n{text}\n{', '.join(acsa)}\n\n")

##Bi-LSTM

In [ ]:
inputs = Input(shape=(MAXLEN,))

x = Embedding(MAX_FEATURES, EMBED_SIZE, weights=[embedding_matrix], trainable=True)(inputs)
x = SpatialDropout1D(0.35)(x)

x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

output_layer = Concatenate(axis=1)([Dense(units=4, activation='softmax', name=aspect.replace('#', '-').replace('&', '_'))(x) for aspect in aspects])

model_Bi_LSTM = Model(inputs, output_layer)
model_Bi_LSTM.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose=1, patience=3)

history = model_Bi_LSTM.fit(x = X_train_pad, y = y_train_model,
                    batch_size=32,
                    epochs=15,
                    verbose=1,
                    validation_data=(X_dev_pad, y_dev_model))

Epoch 1/15
151/151 [==============================] - 252s 2s/step - loss: 0.0326 - accuracy: 0.0015 - val_loss: 0.0833 - val_accuracy: 0.0019
Epoch 2/15
151/151 [==============================] - 245s 2s/step - loss: 0.0287 - accuracy: 0.0023 - val_loss: 0.0870 - val_accuracy: 6.2422e-04
Epoch 3/15
151/151 [==============================] - 250s 2s/step - loss: 0.0261 - accuracy: 0.0023 - val_loss: 0.0889 - val_accuracy: 0.0025
Epoch 4/15
151/151 [==============================] - 248s 2s/step - loss: 0.0234 - accuracy: 0.0056 - val_loss: 0.0920 - val_accuracy: 0.0056
Epoch 5/15
151/151 [==============================] - 239s 2s/step - loss: 0.0215 - accuracy: 0.0040 - val_loss: 0.0953 - val_accuracy: 0.0025
Epoch 6/15
151/151 [==============================] - 237s 2s/step - loss: 0.0193 - accuracy: 0.0077 - val_loss: 0.0977 - val_accuracy: 0.0162
Epoch 7/15
151/151 [==============================] - 239s 2s/step - loss: 0.0181 - accuracy: 0.0104 - val_loss: 0.1021 - val_accuracy: 0.

In [ ]:
y_pred = model_Bi_LSTM.predict(X_test_pad)
y_pred = y_pred.reshape(len(y_pred), -1, 4)
y_pred_argmax = np.argmax(y_pred, axis=-1)

51/51 [==============================] - 10s 199ms/step


In [ ]:
vlsp_results = []
for index, pred in enumerate(y_pred_argmax):
    sentiments = map(lambda x: replacements[x], pred)
    result = {
        'id': f'#{index + 1}',
        'text': X_test[index],
        'acsa': []
    }
    for aspect, sentiment in zip(aspects, sentiments):
        if sentiment: result['acsa'].append('{'+ aspect + ', ' + sentiment + '}')
    vlsp_results.append(result)
vlsp_results[:3]

[{'id': '#1',
  'text': 'Khách sạn tuyệt vời, mình ở 3n2đ đi chơi không kịp báo dọn phòng nhưng khách sạn đã dọn giúp sạch sẽ tinh tươm trước khi mình về, mình book được giá tốt nữa. Rất hài lòng, nv nhiệt tình dễ thương nhẹ nhàng, có dịp quay lại mình sẽ ở lại đây.',
  'acsa': ['{Hotel#General, positive}',
   '{Hotel#Comfort, positive}',
   '{Rooms#Cleanliness, positive}',
   '{Service#General, positive}']},
 {'id': '#2',
  'text': 'Nhân viên phục vụ dễ thương! Khuyết điểm là khách sạn ít đèn quá tối, rất khó chịu. Vòi sen bồn rửa mặt ngắn quá, rửa mặt xong nước chảy tràn ra tùm lum. Phòng tắm không có bồn. Ăn sáng tạm ổn, nói chung với mức tiền đó thì chấp nhận được',
  'acsa': ['{Room_amenities#Designs_Features, negative}',
   '{Room_amenities#Quality, negative}',
   '{Service#General, positive}',
   '{Food_Drinks#Quality, neutral}']},
 {'id': '#3',
  'text': 'Khách sạn rất gần chợ đêm. Xung quanh khách sạn có nhiều địa điểm ăn uống kể cả ban ngày và ban đêm. Nhân viên lễ tân nhiệt 

In [ ]:
with open('/content/model_Bi_LSTM_v3.txt', 'w', encoding='utf-8') as f:
    for result in vlsp_results:
        id, text, acsa = result.values()
        f.write(f"{id}\n{text}\n{', '.join(acsa)}\n\n")

save dev

In [ ]:
y_pred = model_Bi_LSTM.predict(X_dev_pad)
y_pred = y_pred.reshape(len(y_pred), -1, 4)
y_pred_argmax = np.argmax(y_pred, axis=-1)

51/51 [==============================] - 11s 220ms/step


In [ ]:
vlsp_results = []
for index, pred in enumerate(y_pred_argmax):
    sentiments = map(lambda x: replacements[x], pred)
    result = {
        'id': f'#{index + 1}',
        'text': X_dev[index],
        'acsa': []
    }
    for aspect, sentiment in zip(aspects, sentiments):
        if sentiment: result['acsa'].append('{'+ aspect + ', ' + sentiment + '}')
    vlsp_results.append(result)
vlsp_results[:3]

[{'id': '#1',
  'text': 'Khách sạn mới, sạch sẽ, đầy đủ tiện nghi. Vị trí hơi xa, nên hỗ trợ đưa đón sân bay cho khách.',
  'acsa': ['{Hotel#Designs_Features, positive}',
   '{Hotel#Cleanliness, positive}',
   '{Facilities#General, positive}',
   '{Facilities#Designs_Features, positive}']},
 {'id': '#2',
  'text': 'Phòng sạch sẽ, nhân viên nhiệt tình. Đáng để trải nghiệm',
  'acsa': ['{Hotel#General, positive}',
   '{Rooms#Cleanliness, positive}',
   '{Service#General, positive}']},
 {'id': '#3',
  'text': 'Khách sạn sạch đẹp giá cả ok !',
  'acsa': ['{Hotel#Price, positive}',
   '{Hotel#Designs_Features, positive}',
   '{Hotel#Cleanliness, positive}']}]

In [ ]:
with open('/content/dev_Bi_LSTM_v3.txt', 'w', encoding='utf-8') as f:
    for result in vlsp_results:
        id, text, acsa = result.values()
        f.write(f"{id}\n{text}\n{', '.join(acsa)}\n\n")